# Atelier 5 : Débruitage avec PointCloud - une approche plus frugale

In [1]:
# Imports des bibliothèques utiles
# pour l'IA
import torch
# pour les maths
import numpy as np
# pour afficher des images et des courbes
import matplotlib.pyplot as plt

In [ ]:
# Si on veut sauvegarder le notebook sur sur google drive
# from google.colab import drive
# import os
# drive.mount('/content/drive')

In [2]:
! git clone https://github.com/nanopiero/PREAC.git

Cloning into 'PREAC'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 110 (delta 56), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (110/110), 7.18 MiB | 10.25 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [5]:
! pip install einops

## A. Découverte du problème

In [ ]:
# Ici, on s'emploie à une tâche un peu différente:
# détecter les pixels associés aux disques.
# Il s'agit donc de segmentation sémantique.

import PREAC.utile_Transformers
import importlib  # Import the importlib module
importlib.reload(PREAC.utile_Transformers)  # Reload the module
from PREAC.utile_Transformers import gen_pointnet, voir_batch2D, \
                                     get_random_xy_triplets, plot_triplets

batch_size = 6

N = 1000
M = 1100
images, targets, input_points, target_list, target_points  = gen_pointnet(batch_size, N, M)

# Entrées x : images
fig1 = plt.figure(1, figsize=(36, 6))
voir_batch2D(images, 6, fig1, k=0, min_scale=0, max_scale=1)

# Cibles y : classe par pixel
fig2 = plt.figure(2, figsize=(36, 6))
voir_batch2D(targets, 6, fig2, k=0, min_scale=0, max_scale=1)

In [ ]:
# Seulement, au lieu d'aborder le problème avec des images
# nous allons essayer d'utliser des nuages de points.
# Pour créer ces nuages, nous avons échantilloné densément les zones
# associées aux zones non nulles de l'image d'entrée.
# Notez que le nuage est tridimensionnel, la troisième coordonnée
# étant fournie par la valeur au pixel dans l'image.
# La cble est aussi un nuage de point. L'ltitude vaut 1 pour les pixels
# associés à des disques.

for i in range(batch_size):
  print(i)
  plot_triplets(input_points[i].transpose(0,1).cpu())
  plot_triplets(target_points[i].transpose(0,1).cpu())


## B. Entraînement d'un PointNet

In [8]:
from PREAC.utile_Transformers import PointNetDenseCls, feature_transform_regularizer

pointnet = PointNetDenseCls(k=2, feature_transform=True).cuda()
optimizer = torch.optim.Adam(pointnet.parameters(), lr=0.0005, betas=(0.9, 0.999))
device = torch.device('cuda:0')
pointnet = pointnet.to(device)


# Fonction de coût:
import torch.nn.functional as F
loss_function = F.nll_loss

In [ ]:
batch_size = 32
n_epochs = 50
n_batch_per_epoch = 10
N = 1000
M = 1100


for epoch in range(1,n_epochs):
  print('epoch : ', epoch)
  for batch in range(1,n_batch_per_epoch):
    _, _, input_points, target_list, _ = gen_pointnet(batch_size, N, M)

    # On n'utilise que les points :
    input_points = input_points.to(device)
    target_list = target_list.to(device)

    # Init de l'optimizer
    optimizer.zero_grad()

    proba_pred_list, _ , trans_feat = pointnet(input_points)
    proba_pred_list = proba_pred_list.transpose(1,2)
    loss = loss_function(proba_pred_list, target_list)

    # un peu de régularisation ici :
    loss += feature_transform_regularizer(trans_feat) * 0.001


    loss.backward()
    optimizer.step()
    pred_list = proba_pred_list.data.max(1)[1]
    correct = pred_list.eq(target_list.data).cpu().sum()
    print('[%d: %d] train loss: %f accuracy: %f' % (epoch, batch, loss.item(), correct.item()/float(batch_size * pred.size(2))))




In [ ]:
#snippet pour le tracé des solutions :
_, _, input_points, _ , target_points  = gen_pointnet(6, N, M)

# Il faut construire les prédictions.
proba_pred_list, _, _ = pointnet(input_points.to(device))
pred_list = proba_pred_list.transpose(1,2).max(1)[1].cpu().unsqueeze(1)

preds_points = torch.cat((input_points[:,:2,:], preds_list), dim=1)

# Si l'on veut comparer la cible à la sortie :
# xeq = torch.cat((input_points[:,:2,:], preds_list.max(1)[1].eq(uy).unsqueeze(1)), dim = 1)


for i in range(6):
  print(i)
  plot_triplets(input_points[i].transpose(0,1))
  plot_triplets(target_points[i].transpose(0,1))
  plot_triplets(preds_points[i].transpose(0,1).cpu().detach())


**Questions intéressantes** : \\
En quoi cette approche est-elle plus frugale ?
Le résultat est-il satisfaisant ? \\
Quel est le problème ?

## C. Entraînement d'un PointNet++

In [31]:
# On test avec pointNet 2:
! git clone https://github.com/yanx27/Pointnet_Pointnet2_pytorch.git

Cloning into 'Pointnet_Pointnet2_pytorch'...
remote: Enumerating objects: 842, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 842 (delta 7), reused 12 (delta 5), pack-reused 826
Receiving objects: 100% (842/842), 68.77 MiB | 32.56 MiB/s, done.
Resolving deltas: 100% (485/485), done.


In [32]:
! ls

Pointnet_Pointnet2_pytorch  PREAC  sample_data


In [37]:
import os
os.chdir('Pointnet_Pointnet2_pytorch')
import sys
sys.path.append('/content/Pointnet_Pointnet2_pytorch')
import importlib
MODEL = importlib.import_module('models.pointnet2_sem_seg_msg')

In [39]:
# def weights_init(m):
#     classname = m.__class__.__name__
#     if classname.find('Conv2d') != -1:
#         torch.nn.init.xavier_normal_(m.weight.data)
#         torch.nn.init.constant_(m.bias.data, 0.0)
#     elif classname.find('Linear') != -1:
#         torch.nn.init.xavier_normal_(m.weight.data)
#         torch.nn.init.constant_(m.bias.data, 0.0)
def inplace_relu(m):
    classname = m.__class__.__name__
    if classname.find('ReLU') != -1:
        m.inplace=True

# def bn_momentum_adjust(m, momentum):
#     if isinstance(m, torch.nn.BatchNorm2d) or isinstance(m, torch.nn.BatchNorm1d):
#         m.momentum = momentum

In [42]:
NUM_CLASSES = 2

def inplace_relu(m):
    classname = m.__class__.__name__
    if classname.find('ReLU') != -1:
        m.inplace=True

pointnet2 = MODEL.get_model(NUM_CLASSES).cuda()
pointnet2.apply(inplace_relu)
criterion = MODEL.get_loss().cuda()


optimizer = torch.optim.Adam(
    pointnet2.parameters(),
    lr=0.001,
    betas=(0.9, 0.999),
    eps=1e-08
)


In [ ]:
batch_size = 16
N = 500
M = 600
n_epochs = 50

for epoch in range(1,n_epochs):
  print('epoch : ', epoch)
  for batch in range(1,n_batch_per_epoch):
    _, _, input_points, target_list, _ = gen_pointnet(batch_size, N, M)

    input_points = input_points.cuda()
    target_list = target_list.cuda()

    # On se rapporte à des features tridimensionnel pour pourvoir appliquer le
    # modèle défini dans le github de yanx27 (on n'a pas la main sur cet aspect)
    input_points = torch.cat((input_points, input_points, input_points), dim = 1)

    optimizer.zero_grad()
    seg_pred, trans_feat = pointnet2(input_points)

    # Méthode de comparaison altenrantive à celle utilisée au B:
    # on applatit les vecteurs avant comparaison
    seg_pred = seg_pred.contiguous().view(-1, NUM_CLASSES)
    target = target_list.view(-1, 1)[:, 0]
    weights = None
    loss = F.nll_loss(seg_pred, target) #, weight=weight)
    loss.backward()
    optimizer.step()

    # pred = pred.transpose(1,2)
    # loss = F.nll_loss(pred, uy)
    # print(trans_feat)
    # loss += feature_transform_regularizer(trans_feat) * 0.001


    pred_choice = seg_pred.data.max(1)[1]
    correct = pred_choice.eq(target.data.view(-1, 1)[:, 0]).cpu().sum()
    print('[%d: %d] train loss: %f accuracy: %f' % (epoch, batch, loss.item(), correct.item()/float(seg_pred.size(0))))





In [ ]:
#snippet pour le tracé des solutions :
_, _, input_points, _ , target_points  = gen_pointnet(6, N, M)

# Il faut construire les prédictions.
input_points = input_points.to(device)

input_points = torch.cat((input_points, input_points, input_points), dim = 1)
proba_pred_list, _ = classifier(input_points)

# Retour sur CPU
input_points = input_points.cpu()
pred_list = proba_pred_list.transpose(1,2).max(1)[1].cpu().unsqueeze(1)

pred_points = torch.cat((input_points[:,:2,:], pred_list), dim=1)

# Si l'on veut comparer la cible à la sortie :
# xeq = torch.cat((input_points[:,:2,:], preds_list.max(1)[1].eq(uy).unsqueeze(1)), dim = 1)


for i in range(6):
  print(i)
  plot_triplets(input_points[i].transpose(0,1))
  plot_triplets(target_points[i].transpose(0,1))
  plot_triplets(pred_points[i].transpose(0,1).cpu().detach())

**Question intéressante** : \\
Le résultat est-il satisfaisant ? \\
Que se passe-t-il si l'on joue sur les nombres de points présentés au réseau.